In [1]:
import gym
import myrl
from myrl.buffers import ReplayBuffer, PrioritizedReplayBuffer
from myrl.environments import Envs
from myrl.value_functions import ValueFunctionMLP, polyak
from myrl.policies import LinearPolicy, RandomPolicy
from myrl.visualizer import showit
from myrl.utils import normal_noise, ExperimentWriter
import torch
import copy
from collections import deque
import numpy as np
import torch.nn as nn


z = 0
envname = 'Pendulum-v0'
env = gym.make(envname)
envs = Envs(envname, 3)
random_policy = RandomPolicy(env).act
obsdim = env.observation_space.shape[0]
adim = env.action_space.shape[0]
wll = ExperimentWriter('tb/pentd3')

device = torch.device('cuda' if torch.cuda.is_available() or 0 else 'cpu')
device

device(type='cuda')

In [8]:
rbuff = ReplayBuffer(max_len=100000)

maxer = LinearPolicy(obsdim, 32, adim)
q1 = ValueFunctionMLP(obsdim+adim, 64, 32, 1)
q2 = ValueFunctionMLP(obsdim+adim, 64, 32, 1)
tq1 = copy.deepcopy(q1)
tq2 = copy.deepcopy(q2)
opt = torch.optim.Adam(list(q1.parameters())+list(q2.parameters()), lr=1e-3)
mopt = torch.optim.Adam(maxer.parameters(), lr=1e-3)

In [9]:
def chck(pi):
    import numpy as np 
    nes = np.expand_dims(env.observation_space.sample(), axis=0)
    a, (d, sm, h)=pi.act(nes)
    return d
def clip_grad_norm_(module, max_grad_norm):
    nn.utils.clip_grad_norm_([p for g in module.param_groups for p in g["params"]], max_grad_norm)

In [11]:
wll.new()
writer = wll.writer
for i in range(1):
    eps = 10000
    gamma = 0.95
    bsize = 128
    warmup = 5000
    maxeract = lambda obs: maxer.act(obs, std=0.3)

    for ep in range(0, eps):

        for i in range(1):
            pi = random_policy if len(rbuff)<warmup else maxeract
            oldobs, a, r, obs, d, _, _, _ = envs.rollout(pi, debug=0)
            rbuff.add(oldobs, a, r, obs, d)
        writer.add_scalar("multi/rew", r.mean().item(), ep)
        rreal = r.mean().item()
        

        if len(rbuff) < bsize:
            print(len(rbuff), end='\r')
            continue  

        for put in range(20):
            # print(len(rbuff), bsize)
            # print(len(rbuff.deqs[0]))
            oldobs, a, r, obs, d = rbuff.get(bsize)


            for optstep in range(2):
                amax = maxer(obs).detach()
                amax += normal_noise(amax, 0.08)
                in1 = torch.cat((obs, amax), dim=-1)
                in2 = torch.cat((oldobs, a), dim=-1)
                target = r + gamma*(1-d*0)*torch.min(tq1(in1), tq2(in1))[0]
                td_error = ((target - q1(in2))**2) + ((target - q2(in2))**2)
                td = td_error.mean()
                opt.zero_grad()
                td.backward()
                opt.step()
            # rbuff.add(oldobs, a, r, obs, d)#, vals=-td_error)
            
            if not put%2 or 1:
                for optstep in range(4):
                    amax = maxer(obs)
                    in1 = torch.cat((obs, amax), dim=-1)
                    maxer_loss = -q1(in1).mean()
                    mopt.zero_grad()
                    maxer_loss.backward()
                    clip_grad_norm_(mopt, 0.5)
                    mopt.step()

            polyak(q1, tq1, 1-1/70)
            polyak(q2, tq2, 1-1/70)

        writer.add_scalar("multi/maxer_loss", maxer_loss.item(), ep)
        writer.add_scalar("multi/td_q", td.item(), ep)

        if ep%20==0:
            print(ep, maxer_loss.item(), r.mean().item(), td.item(), len(rbuff), chck(maxer).item(), rreal, wll.s+str(wll.z))
        if ep%200==0:
            showit(env, maxer.act, )
            env.close()


0 133.97738647460938 -6.336296081542969 8.408854484558105 100000 -1.0595449209213257 -6.513232707977295 tb/pentd36
100 135.8064727783203 -6.703187942504883 34.17329788208008 100000 -1.3588658571243286 -7.248016834259033 tb/pentd36
120 133.97274780273438 -7.099801540374756 37.11856460571289 100000 -1.2055333852767944 -7.416893482208252 tb/pentd36
140 140.18563842773438 -7.476479530334473 3.5062918663024902 100000 -1.0993207693099976 -7.140685081481934 tb/pentd36
160 136.23696899414062 -7.02285099029541 76.46674346923828 100000 0.4909179210662842 -7.197425365447998 tb/pentd36
180 138.33795166015625 -6.810129642486572 69.14186096191406 100000 -1.690279483795166 -6.618500232696533 tb/pentd36
200 136.7490234375 -6.778636932373047 3.575767993927002 100000 -0.07536043971776962 -7.258176326751709 tb/pentd36
198 /2000220 140.67257690429688 -6.941531181335449 1.0943195819854736 100000 0.14448195695877075 -7.555851459503174 tb/pentd36
240 141.18626403808594 -7.236208915710449 0.9635403156280518 1

KeyboardInterrupt: 